In [1]:
!pip install selenium
!pip install webdriver-manager
!apt update
!apt install chromium-chromedriver
# !pip install --upgrade torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 58.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 30.7 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]                           
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]                
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]                   
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [70.9 kB]        
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Pack

In [2]:
# !pip install --upgrade torch transformers

In [3]:
from os import link
import pandas as pd

In [4]:
from os import link
import pandas as pd
from datetime import datetime, timedelta
import time
import random
from urllib.parse import urljoin
import re
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

class NewsParser:
    def __init__(self, sources_config):
        """
        Initialize with a list of news sources configurations.

        Each source config should be a dictionary with the following fields:
        - url: Base URL of the news site
        - source_name: Name of the news source
        - entries_selector: CSS selector for news entry items on the main page
        - has_pagination: Boolean indicating if the source has pagination
        - pagination_selector: CSS selector for the next page link (if has_pagination is True)
        - title_selector: CSS selector for the news title
        - date_selector: CSS selector for the news publication date
        - content_selector: CSS selector for the full news content
        - date_format: Format string for parsing the date (e.g., "%Y-%m-%d %H:%M:%S")
        - js_rendered: Boolean indicating if the site requires JavaScript rendering
        """
        self.sources_config = sources_config
        self.driver = None

    def initialize_driver(self):
        """Initialize Selenium WebDriver."""
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920,1080")
        options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        driver = webdriver.Chrome(options=options)
        driver.set_page_load_timeout(30)
        return driver

    def get_soup_from_selenium(self, url, wait_time=10):
        """Get BeautifulSoup object from URL using Selenium for JavaScript rendering."""
        if self.driver is None:
            self.driver = self.initialize_driver()

        try:
            self.driver.get(url)
            # Wait for the page to load completely
            time.sleep(wait_time)

            # Get the page source after JavaScript execution
            page_source = self.driver.page_source
            return BeautifulSoup(page_source, 'html.parser')
        except Exception as e:
            print(f"Error fetching {url} with Selenium: {e}")
            return None

    def parse_date(self, date_text, date_format):
        """Parse date string to datetime object using the provided format."""
        try:
            if date_format:
                return datetime.strptime(date_text.strip(), date_format)

            # For special cases in Russian
            if "сегодня" in date_text.lower():
                today = datetime.now()
                time_part = re.search(r'в\s+(\d+):(\d+)', date_text)
                if time_part:
                    hours, minutes = map(int, time_part.groups())
                    return datetime(today.year, today.month, today.day, hours, minutes)
                return today

            elif "вчера" in date_text.lower():
                yesterday = datetime.now() - timedelta(days=1)
                time_part = re.search(r'в\s+(\d+):(\d+)', date_text)
                if time_part:
                    hours, minutes = map(int, time_part.groups())
                    return datetime(yesterday.year, yesterday.month, yesterday.day, hours, minutes)
                return yesterday

            # Default case - check for relative time
            if "час" in date_text.lower():
                hours = int(re.search(r'\d+', date_text).group())
                return datetime.now() - timedelta(hours=hours)
            elif "день" in date_text.lower() or "дня" in date_text.lower() or "дней" in date_text.lower():
                days = int(re.search(r'\d+', date_text).group())
                return datetime.now() - timedelta(days=days)
            elif "минут" in date_text.lower():
                minutes = int(re.search(r'\d+', date_text).group())
                return datetime.now() - timedelta(minutes=minutes)

            # Try common Russian date formats
            for fmt in ["%d.%m.%Y", "%d.%m.%Y %H:%M", "%d %B %Y", "%d %B %Y %H:%M"]:
                try:
                    return datetime.strptime(date_text.strip(), fmt)
                except ValueError:
                    continue

            # Return current date as fallback
            print(f"Could not parse date: {date_text}")
            return datetime.now()
        except Exception as e:
            print(f"Error parsing date '{date_text}': {e}")
            return datetime.now()

    def extract_article_data(self, article_url, source_config):
        """Extract data from a single article page."""
        if source_config.get('js_rendered', False):
            soup = self.get_soup_from_selenium(article_url)
        else:
            response = requests.get(article_url)
            soup = BeautifulSoup(response.text, 'html.parser')

        if not soup:
            return None

        try:
            title_element = soup.select_one(source_config['title_selector'])
            title = title_element.get_text().strip() if title_element else "No title found"
            if title == "No title found":
                return None

            date_element = soup.select_one(source_config['date_selector'])
            date_text = date_element.get_text().strip() if date_element else ""
            date = self.parse_date(date_text, source_config.get('date_format', '')) if date_text else None

            def extract_content(soup, selector):
                content_elements = soup.select(selector)
                if content_elements:
                    txt = ' '.join([p.get_text().strip() for p in content_elements])
                    txt = txt.replace('\xa0', ' ')
                    return txt
                return "No content found"

            content = extract_content(soup, source_config['content_selector'])

            return {
                'title': title,
                'date': date,
                'content': content,
                'url': article_url,
                'source_name': source_config['source_name']
            }
        except Exception as e:
            print(f"Error parsing article {article_url}: {e}")
            return None

    def extract_article_url(self, element, base_url):
        """Extract article URL from an element with better fallback handling."""
        try:
            # First try to find an anchor tag
            link_element = element.find('a')
            if link_element and link_element.get('href'):
                return urljoin(base_url, link_element.get('href'))

            # Try to get href from the element itself
            if element.get('href'):
                return urljoin(base_url, element.get('href'))

            # Look for any element with href attribute
            hrefs = element.select('[href]')
            if hrefs:
                return urljoin(base_url, hrefs[0].get('href'))

            # If element is already an 'a' tag
            if element.name == 'a' and element.get('href'):
                return urljoin(base_url, element.get('href'))

            return None
        except Exception as e:
            print(f"Error extracting article URL: {e}")
            return None

    def handle_pagination(self, source_config, soup, base_url, current_url, page_count):
        """
        Handle pagination with more flexible approach:
        1. If no pagination is configured, scroll down until enough articles are found
        2. If pagination exists, use direct URL pattern if available or find "load more" button

        Returns:
            tuple: (new_current_url, new_page_count, should_continue)
        """
        # Case 1: No pagination configured - scroll down for more content
        if not source_config.get('has_pagination', False):
            if source_config.get('js_rendered', False) and self.driver:
                try:
                    # Scroll down to load more content
                    self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(20)  # Wait for content to load

                    # Check if new content was loaded
                    print(self.driver.page_source)
                    new_soup = BeautifulSoup(self.driver.page_source, 'html.parser')
                    print(len(new_soup.select(source_config['entries_selector'])), len(soup.select(source_config['entries_selector'])))
                    if len(new_soup.select(source_config['entries_selector'])) > len(soup.select(source_config['entries_selector'])):
                        return current_url, page_count, True
                    else:
                        return current_url, page_count, False
                except Exception as e:
                    print(f"Error scrolling for more content: {e}")
                    return current_url, page_count, False
            else:
                # No pagination and not JS rendered - can't get more content
                return current_url, page_count, False

        # Case 2: Check if direct pagination URL is specified
        if source_config.get('pagination_direct_url', False):
            pattern = source_config.get('pagination_postfix', '?page={}')
            if '{}' in pattern:
                next_url = f"{base_url}/{pattern.format(page_count + 1)}"
                print(next_url)
            else:
                next_url = f"{base_url}/{pattern}"
            page_count += 1
            return next_url, page_count, True

        # Case 3: Standard pagination handling based on selectors
        if source_config.get('js_rendered', False) and self.driver:
            try:
                # Try to find "Load More" button if that's the pagination type
                if source_config.get('pagination_type') == 'load_more' and 'pagination_selector' in source_config:
                    try:
                        next_page_element = WebDriverWait(self.driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, source_config['pagination_selector']))
                        )
                        if next_page_element:
                            next_page_element.click()
                            time.sleep(2)  # Wait for content to load
                            return current_url, page_count, True
                    except Exception as e:
                        print(f"Error clicking load more button: {e}")

                # Try to find numbered pagination
                if 'pagination_selector' in source_config:
                    try:
                        next_page_elements = self.driver.find_elements(By.CSS_SELECTOR, source_config['pagination_selector'])
                        if next_page_elements:
                            for element in next_page_elements:
                                # Look for next page number or "next" button
                                href = element.get_attribute('href')
                                if href:
                                    if str(page_count + 1) in href or "next" in element.text.lower():
                                        next_url = href
                                        page_count += 1
                                        return next_url, page_count, True
                    except NoSuchElementException:
                        pass

                # Try URL pattern fallbacks
                if "?page=" in current_url:
                    base_part = current_url.split('?page=')[0]
                    next_url = f"{base_part}?page={page_count + 1}"
                elif "/page/" in current_url:
                    base_part = current_url.split('/page/')[0]
                    next_url = f"{base_part}/page/{page_count + 1}"
                else:
                    # Add appropriate pagination suffix based on common patterns
                    if "?" in current_url:
                        next_url = f"{current_url}&page={page_count + 1}"
                    else:
                        next_url = f"{current_url}?page={page_count + 1}"

                page_count += 1
                return next_url, page_count, True

            except Exception as e:
                print(f"Error handling JS pagination: {e}")
                return current_url, page_count, False
        else:
            # Handle non-JS rendered pagination
            if 'pagination_selector' in source_config:
                next_page_element = soup.select_one(source_config['pagination_selector'])
                if next_page_element and next_page_element.get('href'):
                    next_url = urljoin(base_url, next_page_element.get('href'))
                    page_count += 1
                    return next_url, page_count, True

                # Try to find next page by pattern
                pagination_elements = soup.select(source_config['pagination_selector'])
                for element in pagination_elements:
                    if str(page_count + 1) in element.text or (element.get('href') and str(page_count + 1) in element.get('href')):
                        next_url = urljoin(base_url, element.get('href'))
                        page_count += 1
                        return next_url, page_count, True

        # No pagination options worked
        return current_url, page_count, False

    def parse_source(self, source_config):
        """Parse a single news source according to its configuration."""
        if 'base_url' in source_config:
            base_url = source_config['base_url']
        else:
            base_url = source_config['url']
        current_url = source_config['url']
        articles_data = []
        cutoff_date = datetime.now() - timedelta(days=2)
        page_count = 1
        max_attempts = 10  # Maximum pagination attempts

        while len(articles_data) < 50 and page_count <= max_attempts:
            print(f"Processing page {page_count} from {source_config['source_name']}")

            if source_config.get('js_rendered', False):
                soup = self.get_soup_from_selenium(current_url)
            else:
                response = requests.get(current_url)
                soup = BeautifulSoup(response.text, 'html.parser')

            if not soup:
                break

            entry_elements = soup.select(source_config['entries_selector'])
            if not entry_elements:
                print(f"No entries found on {current_url}")

                # If no entries but JS rendered, try scrolling
                if source_config.get('js_rendered', False) and self.driver:
                    try:
                        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(3)
                        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
                        entry_elements = soup.select(source_config['entries_selector'])
                        if not entry_elements:
                            break
                    except Exception as e:
                        print(f"Error during scroll attempt: {e}")
                        break
                else:
                    break

            # Process articles on the current page
            continue_parsing = True
            if source_config['has_pagination'] or len(entry_elements) >= 50:


                for element in entry_elements:
                    # Extract link to full article with improved extraction
                    article_url = self.extract_article_url(element, base_url)
                    if not article_url:
                        continue

                    # Delay between requests to avoid overloading the server
                    time.sleep(random.uniform(1, 3))

                    article_data = self.extract_article_data(article_url, source_config)
                    if article_data and article_data['date']:
                        if article_data['date'] < cutoff_date:
                            continue_parsing = False
                            break

                        articles_data.append(article_data)
                        print(f"Article added: {article_data['title']}")

                        if len(articles_data) >= 50:
                            break

                # Check if we should stop due to date cutoff or article limit
                if not continue_parsing or len(articles_data) >= 30:
                    break

            # Handle pagination with the updated flexible approach
            current_url, page_count, should_continue = self.handle_pagination(
                source_config, soup, base_url, current_url, page_count)
            print(current_url, page_count, should_continue)
            if not should_continue:
                break

        return articles_data

    def parse_all_sources(self):
        """Parse all news sources and return the data as a DataFrame."""
        all_articles = []

        try:
            for source_config in self.sources_config:
                try:
                    print(f"\nParsing news from: {source_config['source_name']}")
                    source_articles = self.parse_source(source_config)
                    all_articles.extend(source_articles)
                    print(f"Collected {len(source_articles)} articles from {source_config['source_name']}")
                except Exception as e:
                    print(f"Error parsing source {source_config['source_name']}: {e}")
        finally:
            # Make sure to close the driver
            if self.driver:
                self.driver.quit()

        # Convert to DataFrame
        if all_articles:
            df = pd.DataFrame(all_articles)
            return df
        else:
            return pd.DataFrame(columns=['title', 'date', 'content', 'url', 'source_name'])

# Example usage
if __name__ == "__main__":
    import requests

    # Define the RIAMO news source configuration
    sources_config = [
        {
            'url': 'https://riamo.ru/category/proisshestviya/',
            'source_name': 'RIAMO',
            'entries_selector': 'article._1i8pq983',
            'has_pagination': True,
            'pagination_selector': 'a[href*="?page="]',
            'title_selector': 'h1.dx3d6b0',
            'date_selector': 'time.xtn0kl0',
            'content_selector': 'main p',  # This will need to be verified
            'js_rendered': True,  # Set to True to use Selenium for JavaScript rendering
            'pagination_direct_url':'?page={}'
        }
    #     {
    #     'url': 'https://78.ru/news/proisshestviya',
    #     'source_name': '78.ru',
    #     'entries_selector': 'a.news-feed-timeline-item_item__hFnM1',  # Selector for news items
    #     'has_pagination': False,
    #     'title_selector': 'h1.heading',  # Title selector from the news item
    #     'date_selector': '.author-and-date_containerDate__EJTrp',  # Date selector
    #     'content_selector': 'div.publication__body',  # This needs to be verified on article page
    #     'js_rendered': True,  # Set to True to use Selenium for JavaScript rendering
    #     'base_url':'https://78.ru/news/'
    # }
    ]

    # Create parser and get data
    parser = NewsParser(sources_config)
    news_df = parser.parse_all_sources()

    # Save to CSV
    news_df.to_csv('riamo_news.csv', index=False, encoding='utf-8')
    print(f"Saved {len(news_df)} articles to riamo_news.csv")

    # Display the first few rows
    print("\nSample data:")
    print(news_df.head())


Parsing news from: RIAMO
Processing page 1 from RIAMO
Article added: На территорию Европейского экономического союза хотели ввезти 580 кг взрывчатки
Article added: Лесной пожар ликвидировали в Шатуре
Article added: В Сиднее за дебош на борту самолета задержали чиновника из Иордании
Article added: Стало известно, как себя чувствует подранный медведем мужчина в Подмосковье
Article added: В Ленобласти нашли тело четвертой жертвы «поймавшего белку» мужчины
https://riamo.ru/category/proisshestviya//?page=2
https://riamo.ru/category/proisshestviya//?page=2 2 True
Processing page 2 from RIAMO
Article added: Прокуратура: пятилетний мальчик выпал из окна в Москве
Article added: Стая агрессивных бездомных собак напала на двух школьниц под Пермью
Article added: Мужчину из Кузбасса арестовали за предложение сжечь депутата Госдумы Ямпольскую
Article added: У подростка из России случился инфаркт в Таиланде из-за ОРВИ
Article added: Многодетный папа надругался над приемным ребенком в машине в Подмос

In [5]:
news_df.head()

,title,date,content,url,source_name
0,На территорию Европейского экономического союз...,2025-04-06 20:50:00,Фото - © Telegram-канал Государственного тамож...,https://riamo.ru/news/proisshestviya/na-territ...,RIAMO
1,Лесной пожар ликвидировали в Шатуре,2025-04-06 20:06:00,Фото - © Комитет лесного хозяйства Московской ...,https://riamo.ru/news/proisshestviya/lesnoj-po...,RIAMO
2,В Сиднее за дебош на борту самолета задержали ...,2025-04-06 20:05:00,Фото - © Unsplash.com Подписывайтесь на РИАМО:...,https://riamo.ru/news/proisshestviya/v-sidnee-...,RIAMO
3,"Стало известно, как себя чувствует подранный м...",2025-04-06 18:34:00,Фото - © Unsplash.com Подписывайтесь на РИАМО:...,https://riamo.ru/news/proisshestviya/stalo-izv...,RIAMO
4,В Ленобласти нашли тело четвертой жертвы «пойм...,2025-04-06 16:58:00,Фото - © Telegram-канал Следственного управлен...,https://riamo.ru/news/proisshestviya/v-lenobla...,RIAMO


In [6]:
# !pip install --upgrade torch transformers torchvision

In [7]:
news_df

,title,date,content,url,source_name
0,На территорию Европейского экономического союз...,2025-04-06 20:50:00.000000,Фото - © Telegram-канал Государственного тамож...,https://riamo.ru/news/proisshestviya/na-territ...,RIAMO
1,Лесной пожар ликвидировали в Шатуре,2025-04-06 20:06:00.000000,Фото - © Комитет лесного хозяйства Московской ...,https://riamo.ru/news/proisshestviya/lesnoj-po...,RIAMO
2,В Сиднее за дебош на борту самолета задержали ...,2025-04-06 20:05:00.000000,Фото - © Unsplash.com Подписывайтесь на РИАМО:...,https://riamo.ru/news/proisshestviya/v-sidnee-...,RIAMO
3,"Стало известно, как себя чувствует подранный м...",2025-04-06 18:34:00.000000,Фото - © Unsplash.com Подписывайтесь на РИАМО:...,https://riamo.ru/news/proisshestviya/stalo-izv...,RIAMO
4,В Ленобласти нашли тело четвертой жертвы «пойм...,2025-04-06 16:58:00.000000,Фото - © Telegram-канал Следственного управлен...,https://riamo.ru/news/proisshestviya/v-lenobla...,RIAMO
5,Прокуратура: пятилетний мальчик выпал из окна ...,2025-04-06 15:38:00.000000,Фото - © Telegram-канал Прокуратуры Москвы Под...,https://riamo.ru/news/proisshestviya/prokuratu...,RIAMO
6,Стая агрессивных бездомных собак напала на дву...,2025-04-06 15:35:00.000000,Фото - © Щеголева Ольга / Фотобанк Лори Подпис...,https://riamo.ru/news/proisshestviya/staja-agr...,RIAMO
7,Мужчину из Кузбасса арестовали за предложение ...,2025-04-06 15:21:00.000000,"Фото - © РИАМО, Николай Корешков Подписывайтес...",https://riamo.ru/news/proisshestviya/muzhchinu...,RIAMO
8,У подростка из России случился инфаркт в Таила...,2025-04-06 15:09:00.000000,Фото - © Telegram-канал SHOT Подписывайтесь на...,https://riamo.ru/news/proisshestviya/u-podrost...,RIAMO
9,Многодетный папа надругался над приемным ребен...,2025-04-06 14:39:00.000000,Фото - © Pixabay.com Подписывайтесь на РИАМО: ...,https://riamo.ru/news/proisshestviya/mnogodetn...,RIAMO


In [8]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from tqdm import tqdm
import numpy as np


model_name = "surdan/LaBSE_ner_nerel"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Specify device correctly without using "auto" which can cause issues
device = 'cuda:1' if torch.cuda.is_available() else -1
ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=device)


def extract_relevant_sentences(texts):
    """Extract sentences containing location-related entities using NER for multiple texts"""
    global ner
    shortened_texts = []
    with torch.no_grad():
        for text in texts:
            # Split text into sentences (simple version)
            sentences = [s.strip() for s in text.split('.') if s.strip()]
    
            relevant_sentences = []
            for sent in sentences:
                entities = ner(sent)
                for ent in entities:
                    if ent['entity_group'] in ['CITY', 'LOC', 'ADDR', 'LOCATION', 'ADDRESS', 'ORG']:
                        relevant_sentences.append(sent)
                        break
    
            shortened_texts.append('. '.join(relevant_sentences) + '.' if relevant_sentences else "")
    return shortened_texts

# Initialize pipelines - fix model loading issues
with torch.no_grad():
    gen_pipe = pipeline(
        "text-generation",
        model="sambanovasystems/SambaLingo-Russian-Chat",
        # Don't use device_map="auto" to avoid disk offloading issues
        device='cuda:0' if torch.cuda.is_available() else -1,
        model_kwargs={"torch_dtype": torch.float16 if torch.cuda.is_available() else torch.float32}
    )

# Process in batches of 3
batch_size = 1
addresses = []

# Assuming news_df is defined elsewhere in your code
# If news_df is not defined, you need to define it before using it
try:
    for i in tqdm(range(0, len(news_df), batch_size)):
        batch = news_df.iloc[i:i+batch_size]
        batch_texts = batch['content'].tolist()

        # First extract relevant sentences for the whole batch
        shortened_texts = extract_relevant_sentences(batch_texts)
        # shortened_texts = batch_texts
        # Process each shortened text in the batch
        batch_addresses = []
        for shortened_text in shortened_texts:
            if not shortened_text:
                batch_addresses.append("")
                continue

            prompt = f"""
            Найди в этом тексте адрес происшествия и напиши его. 
            Ответ должен быть кратким, только адрес или строка нет адреса. Пример: улица Островского, Москва.

            Текст:
            {shortened_text}
            """

            messages = [{"role": "user", "content": prompt}]
            with torch.no_grad():
                prompt_template = gen_pipe.tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True
                )
                outputs = gen_pipe(prompt_template, max_new_tokens=50)[0]["generated_text"]
            model_response = outputs.split(prompt_template)[-1].strip()
            print(model_response, shortened_text)
            batch_addresses.append(model_response)

        addresses.extend(batch_addresses)

    news_df['Location'] = addresses

except NameError:
    print("Error: news_df is not defined. Please define your dataframe before running this code.")

tokenizer_config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/521k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/511M [00:00<?, ?B/s]

...

Улица Островского, Москва. Подписывайтесь на РИАМО: В подмосковной деревне Пучково 5 апреля загорелась двухэтажная баня, момент пожара попал на видео.


 95%|█████████▍| 36/38 [00:51<00:03,  1.82s/it]

Фирановское шоссе, Москва. Подписывайтесь на РИАМО: Межрайонная природоохранная прокуратура Москвы вместе с прокуратурой Зеленоградского административного округа начали проверку из-за загрязнения реки Сходни, а также ее водоохранной зоны. Во время проверки территории на Фирановском шоссе обнаружили, что в прибрежную зону реки Сходня сбросили бетонные отходы. Сходня — река в Подмосковье и Москве. Она — левый приток Москвы-реки.


100%|██████████| 38/38 [00:52<00:00,  1.39s/it]

Улица Островского, Москва. Полиция уже нашла и задержала двух подозреваемых в совершении убийства 14-летней Влады в селе Тумкино Ульяновской области.


In [9]:
news_df = news_df[news_df.Location != 'Улица Островского, Москва.']

In [10]:
import requests

def yandex_geocode(address, api_key, lang='ru_RU', limit=1):
    """
    Geocode an address using Yandex Maps API

    Args:
        address (str): Address to geocode
        api_key (str): Yandex API key
        lang (str): Response language (e.g., 'ru_RU', 'en_US')
        limit (int): Max number of results

    Returns:
        dict: API response in JSON format
    """
    base_url = "https://geocode-maps.yandex.ru/1.x/"

    params = {
        'apikey': api_key,
        'geocode': address,
        'lang': lang,
        'format': 'json',
        'results': limit
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise exception for HTTP errors
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None

In [11]:
lat, lon = [], []
for i, row in news_df.iterrows():
    loc = row.loc['Location']
    api_key = "d4368419-b000-4d7d-b66a-6a3dc7da9e45"
    address = loc
    result = yandex_geocode(address, api_key)
    if result:
        try:
            pos = result['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']
            longitude, latitude = pos.split()
            lat.append(latitude)
            lon.append(longitude)
        except (KeyError, IndexError):
            print("No results found in the response")
            lat.append(None)
            lon.append(None)
    else:
        print("Full response:", result)
        lat.append(None)
        lon.append(None)

news_df['longitude'] = lon
news_df['latitude'] = lat

Error making request: 400 Client Error: Bad Request for url: https://geocode-maps.yandex.ru/1.x/?apikey=d4368419-b000-4d7d-b66a-6a3dc7da9e45&geocode=&lang=ru_RU&format=json&results=1
Full response: None
Error making request: 400 Client Error: Bad Request for url: https://geocode-maps.yandex.ru/1.x/?apikey=d4368419-b000-4d7d-b66a-6a3dc7da9e45&geocode=&lang=ru_RU&format=json&results=1
Full response: None
Error making request: 400 Client Error: Bad Request for url: https://geocode-maps.yandex.ru/1.x/?apikey=d4368419-b000-4d7d-b66a-6a3dc7da9e45&geocode=&lang=ru_RU&format=json&results=1
Full response: None


<ipython-input-11-bfdd0cf7ef5a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['longitude'] = lon
<ipython-input-11-bfdd0cf7ef5a>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['latitude'] = lat


In [12]:
news_df

,title,date,content,url,source_name,Location,longitude,latitude
4,В Ленобласти нашли тело четвертой жертвы «пойм...,2025-04-06 16:58:00.000000,Фото - © Telegram-канал Следственного управлен...,https://riamo.ru/news/proisshestviya/v-lenobla...,RIAMO,"Липная Горка, Тихвинский район, Ленинградская ...",33.183389,59.584026
5,Прокуратура: пятилетний мальчик выпал из окна ...,2025-04-06 15:38:00.000000,Фото - © Telegram-канал Прокуратуры Москвы Под...,https://riamo.ru/news/proisshestviya/prokuratu...,RIAMO,"Улица Театральная, Щербинка, Москва.",37.564608,55.502243
9,Многодетный папа надругался над приемным ребен...,2025-04-06 14:39:00.000000,Фото - © Pixabay.com Подписывайтесь на РИАМО: ...,https://riamo.ru/news/proisshestviya/mnogodetn...,RIAMO,"Таракановское шоссе, Солнечногорск, Московская...",36.995669,56.274509
13,Труп зацеперши 2010 года рождения нашли на кры...,2025-04-06 11:31:00.000000,Фото - © Медиасток.рф Подписывайтесь на РИАМО:...,https://riamo.ru/news/proisshestviya/trup-zats...,RIAMO,,None,None
15,Мужчина обстрелял соседа из самодельной пулеме...,2025-04-06 11:23:00.000000,Подписывайтесь на РИАМО: В Алтайском крае сотр...,https://riamo.ru/news/proisshestviya/muzhchina...,RIAMO,"Улица Островского, Москва",37.625612,55.736306
19,Лесной пожар ликвидировали в Люберцах,2025-04-06 20:59:40.755273,Фото - © Комитет лесного хозяйства Московской ...,https://riamo.ru/news/proisshestviya/lesnoj-po...,RIAMO,"Улица Островского, Люберцы, Московская область.",38.005339,55.669681
20,Названа причина тления на крыше Российской гос...,2025-04-06 20:59:57.684193,Фото - © Фотобанк Лори Подписывайтесь на РИАМО...,https://riamo.ru/news/proisshestviya/nazvana-p...,RIAMO,"Улица Воздвиженка, Москва.",37.605715,55.752769
24,В Москве автомобиль врезался в столб и создал ...,2025-04-06 21:01:02.158995,Фото - © Медиасток.рф Подписывайтесь на РИАМО:...,https://riamo.ru/news/proisshestviya/v-moskve-...,RIAMO,"Островского ул., Москва.",37.625612,55.736306
27,В Одессе украинец с ружьем отправился в военко...,2025-04-06 21:01:45.816018,Фото - © Николай Винокуров / Фотобанк Лори Под...,https://riamo.ru/news/proisshestviya/v-odesse-...,RIAMO,"Улица Заньковецкой, Одесса, Украина.",30.711378,46.458492
28,Прокуратура: московские правоохранители раскры...,2025-04-06 21:02:03.252423,Фото - © Фотобанк Лори Подписывайтесь на РИАМО...,https://riamo.ru/news/proisshestviya/prokuratu...,RIAMO,"3-й Волоколамский проезд, Москва.",37.486275,55.802578


In [13]:
news_df.to_csv('test.csv', index=False)

In [14]:
news_df.shape

(15, 8)